In [1]:
import cv2 as cv
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import networkx as nx
from IPython.display import Audio, display
import ipynb.fs.defs.Utils as Utils

In [2]:
def allDone():
    print(" --- finished --- ")
    #display(Audio(url='https://sound.peal.io/ps/audios/000/001/131/original/kon.wav', autoplay=True))

In [3]:
#This class contains the methods and properties related to the features of an image
class ImageFeature:
    def __init__(self, image, index):
        self.image = image
        self.index = index  
        
    #Compute the salient points of the image using SIFT algorithm
    def SIFT(self, save_output = False, output_dir = "output"):
        sift = cv.SIFT_create()
        self.kp, self.des = sift.detectAndCompute(self.image,None)
        self.img_with_sift=cv.drawKeypoints(self.image,self.kp,np.copy(self.image))
        if save_output:
            cv.imwrite(os.path.join(output_dir,f"{self.index+1}.jpg"), self.img_with_sift)

In [4]:
#This class contains the methods and properties related to the problem of feature matching between two images
class Match:
    def __init__(self, image_feature_source, image_feature_destination):
        self.image_feature_source = image_feature_source
        self.image_feature_destination = image_feature_destination
    
    #This function performs feature matching between to images
    def feature_matching(self, threshold, save_output = False, output_dir = "output", noisy = False):
        # FLANN parameters
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)   # or pass empty dictionary
        flann = cv.FlannBasedMatcher(index_params,search_params)
        
        bf = cv.BFMatcher()
        matches = bf.knnMatch(self.image_feature_source.des,
                                 self.image_feature_destination.des,
                                 k=3)
        # Need to draw only good matches, so create a mask
        self.good = []
        for m in matches:
            if m[0].distance < threshold*m[1].distance:
                r_m = np.random.choice(m) if noisy else m[0]
                self.good.append((r_m,r_m))
        draw_params = dict(matchColor = (0,255,0),
                           singlePointColor = (255,0,0),
                           flags = cv.DrawMatchesFlags_DEFAULT)
        self.img_with_match = cv.drawMatchesKnn(self.image_feature_source.image,
                                                self.image_feature_source.kp,
                                                self.image_feature_destination.image,
                                                self.image_feature_destination.kp,
                                                self.good,None,**draw_params)
        if save_output:
            cv.imwrite(os.path.join(output_dir,f"match_{self.image_feature_source.index+1}_{ self.image_feature_destination.index+1}.jpg"),self.img_with_match)
    
    #This function allows to check whether the match between the source and destination images is to be considered a good match
    def check_salient(self, threshold, save_output = False, output_dir = "output"):
        #A threshold is used to define the goodness of the  match
        if len(self.good) > threshold:
            if save_output:
                cv.imwrite(os.path.join(output_dir,f"salient_{self.image_feature_source.index+1}_{ self.image_feature_destination.index+1}.jpg"),self.img_with_match)
            return True
        return False
    
    def mult_and_norm(self,H, points):
        proj_p = np.dot(H,points)
        proj_p = proj_p / proj_p[2,:]
        return proj_p[0:2,:]
    
    #This function allows to estimate the homography between the two images
    def fit_homography(self, RANSACmaxIters = 2000, T_norm = np.eye(3)):
        self.src_pts = np.double([self.image_feature_source.kp[m[0].queryIdx].pt for m in self.good])
        self.dst_pts = np.double([self.image_feature_destination.kp[m[0].trainIdx].pt for m in self.good])
        
        self.src_pts = np.concatenate((self.src_pts, np.ones([self.src_pts.shape[0],1])), axis=1)
        self.dst_pts = np.concatenate((self.dst_pts, np.ones([self.dst_pts.shape[0],1])), axis=1)
        
        
        src_pts_proj = self.mult_and_norm(T_norm,self.src_pts.transpose()).transpose()
        dst_pts_proj = self.mult_and_norm(T_norm,self.dst_pts.transpose()).transpose()
        
        
        self.M, self.mask = cv.findHomography(src_pts_proj, dst_pts_proj, cv.RANSAC, 3*T_norm[0][0], 3.0, maxIters=RANSACmaxIters )
    
    #This function allows to normalize the homography so that the determinant is unitary
    #This makes homographies part of the SL(3) group
    def normalize_homography(self):
        det = np.linalg.det(self.M)
        self.H = self.M/np.cbrt(det)
        new_det = np.linalg.det(self.H)
        
    #Remember: if src is image I and dest is image J, we are finding H from I to J, this is 
    #Hj,i
    

In [5]:
def compute_matches(imgs,
                    T_norm,
                    save_images = False,
                    matching_threshold = 0.6,
                    matches_dir = "matches",
                    matches_th = 20,
                    number_of_matches=1,
                    salient_matches_dir = "salient_matches",
                    sift_dir = "sift",
                    RANSACmaxIters = 2000,
                    noisy_matching = False):
    n = len(imgs)
    image_features =[] #Array that will contain the salient features of each image
    for i in range(0,n):
        image_feature = ImageFeature(imgs[i], i)
        image_feature.SIFT(save_images, sift_dir)
        image_features.append(image_feature)
        
    #Compute the good matches between each pair of images
    matches =[]
    for i in range(0, n-1):
        for j in range(i+1, n):
            for k in range(0,number_of_matches):
                match = Match(image_features[i], image_features[j])
                match.feature_matching(matching_threshold, save_images, matches_dir, noisy = noisy_matching)
                matches.append(match)


    salient_matches = list(filter(lambda x: x.check_salient(matches_th, save_images, salient_matches_dir),matches))
    
    #For every good match, compute also the matches in the reverse order (destination, source) and compute homographies
    total_matches = []
    for match in salient_matches:
        inv_match = Match(match.image_feature_destination, match.image_feature_source)
        inv_match.feature_matching(matching_threshold, save_images, matches_dir, noisy = noisy_matching)
        match.fit_homography(RANSACmaxIters, T_norm)
        match.normalize_homography()
        inv_match.fit_homography(RANSACmaxIters, T_norm)
        inv_match.normalize_homography()
        total_matches.append(match)
        total_matches.append(inv_match)
        
    return total_matches

In [6]:
def get_feature_matches(dataset_name,
                imgs,
                T_norm,
                matching_threshold = 0.6,
                number_of_matches = 1,
                matches_th = 20,
                RANSACmaxIters = 2000,
                save_images = True,
                save_output = True,
                output_dir ="output",
                results_dir = "results",
                noisy_matching = False,
                verbose = True
               ):
        
        sift_dir = os.path.join(results_dir,'sift')
        matches_dir = os.path.join(results_dir,'matches')
        salient_matches_dir = os.path.join(results_dir,'salient_matches')
        Weight_filename = f"W_{dataset_name}.npy"
        
        if save_images:
            shutil.rmtree(results_dir)
            if not os.path.isdir(results_dir):   
                os.mkdir(results_dir)
            if not os.path.isdir(sift_dir):
                os.mkdir(sift_dir)
            if not os.path.isdir(matches_dir):
                os.mkdir(matches_dir)
            if not os.path.isdir(salient_matches_dir):
                os.mkdir(salient_matches_dir)   
            
        n = len(imgs)
        
        matches = compute_matches(imgs = imgs, 
                                  T_norm = T_norm,
                                  save_images= save_images, 
                                  matching_threshold = matching_threshold, 
                                  matches_dir = matches_dir, 
                                  matches_th = matches_th,
                                  number_of_matches = number_of_matches,
                                  salient_matches_dir = salient_matches_dir, 
                                  sift_dir = sift_dir,
                                  RANSACmaxIters = RANSACmaxIters,
                                  noisy_matching = noisy_matching
                                 )
        
        matches_dict = dict()
        weight_matrix = np.zeros([n,n])
        for match in matches:
            i,j = match.image_feature_source.index, match.image_feature_destination.index
            if (i, j) not in matches_dict:
                matches_dict[i, j] = list()
            matches_dict[i, j].append(match.H)
            weight_matrix[i, j] = len(match.good)
        
        if save_output:
            if not os.path.isdir(output_dir):
                os.makedirs(output_dir)
            np.save(os.path.join(output_dir,Weight_filename), weight_matrix)

        if verbose:#pay attention
            allDone()
        
        return matches_dict, weight_matrix
        